## Testing Notebook


In [ ]:
%load_ext autoreload
%autoreload 2

from fibsem import utils, acquire, patterning
from fibsem.structures import BeamType
from pprint import pprint
import matplotlib.pyplot as plt


microscope, settings = utils.setup_session()

In [ ]:
## apply configuration

microscope.apply_configuration()


electron = microscope.get_beam_system_settings(beam_type=BeamType.ELECTRON)
pprint(electron.to_dict())

ion = microscope.get_beam_system_settings(beam_type=BeamType.ION)
pprint(ion.to_dict())



In [ ]:
# flat to beams
microscope.move_flat_to_beam(beam_type=BeamType.ELECTRON)

In [ ]:
microscope.move_flat_to_beam(beam_type=BeamType.ION)

In [ ]:
state = microscope.get_microscope_state()

pprint(state.to_dict())

In [ ]:
# move to state
microscope.set_microscope_state(state)

In [ ]:
## manipulator

microscope.insert_manipulator("PARK")


settings.image.hfw = 400e-6
eb_image, ib_image = acquire.take_reference_images(microscope, settings.image)

fig, ax = plt.subplots(1,2, figsize=(10,5))
ax[0].imshow(eb_image.data, cmap="gray")
ax[1].imshow(ib_image.data, cmap="gray")

plt.show()


In [ ]:

from autolamella.workflows import actions
actions.move_needle_to_liftout_position(microscope)

settings.image.hfw = 400e-6
eb_image, ib_image = acquire.take_reference_images(microscope, settings.image)

fig, ax = plt.subplots(1,2, figsize=(10,5))
ax[0].imshow(eb_image.data, cmap="gray")
ax[1].imshow(ib_image.data, cmap="gray")

plt.show()


In [ ]:
microscope.move_manipulator_corrected(dx=10e-6, dy=10e-6, beam_type=BeamType.ELECTRON)

settings.image.hfw = 400e-6
eb_image, ib_image = acquire.take_reference_images(microscope, settings.image)

fig, ax = plt.subplots(1,2, figsize=(10,5))
ax[0].imshow(eb_image.data, cmap="gray")
ax[1].imshow(ib_image.data, cmap="gray")

plt.show()

microscope.move_manipulator_corrected(dx=10e-6, dy=10e-6, beam_type=BeamType.ION)

settings.image.hfw = 400e-6
eb_image, ib_image = acquire.take_reference_images(microscope, settings.image)

fig, ax = plt.subplots(1,2, figsize=(10,5))
ax[0].imshow(eb_image.data, cmap="gray")
ax[1].imshow(ib_image.data, cmap="gray")

plt.show()

In [ ]:
# chamber camera
image = microscope.acquire_chamber_image()

plt.imshow(image.data, cmap="gray")
plt.show()


In [ ]:
# waffle dry run - full

# serial liftout - prep, mill trench, liftout

In [ ]:
%load_ext autoreload
%autoreload 2

from fibsem.patterning import get_milling_stages
from fibsem.milling import mill_stages, draw_patterns

from fibsem import utils

protocol = {"milling": { 
            "trench": {        
        "application_file": "Si",
        "depth": 1.25e-06,
        "hfw": 0.00018,
        "lamella_height": 2.5e-05,
        "lamella_width": 2.2e-05,
        "milling_current": 50e-12,
        "milling_voltage": 2000,
        "offset": 0.0,
        "size_ratio": 1.0,
        "trench_height": 3.2e-05,
        "type": "Trench",
        "time": 30,
        "milling_channel": "ELECTRON",
            }
}}


microscope, settings = utils.setup_session()
settings.protocol =  protocol

stages = get_milling_stages("trench", settings.protocol["milling"])

from fibsem.milling import draw_patterns

# microscope.connection.imaging.set_active_view(BeamType.ELECTRON.value)  # the ion beam view
# microscope.connection.imaging.set_active_device(BeamType.ELECTRON.value)
# microscope.connection.patterning.set_default_beam_type(BeamType.ELECTRON.value)
# microscope.connection.patterning.clear_patterns()
draw_patterns(microscope, stages[0].pattern.patterns)

# mill_stages(microscope, stages)

In [ ]:
from fibsem.structures import BeamType
microscope.set("voltage", 2000, BeamType.ELECTRON)

#### Parallel Trench Milling

REF: https://www.biorxiv.org/content/10.1101/2024.06.20.599830v1.full.pdf

In [ ]:
%load_ext autoreload
%autoreload 2

import os

import matplotlib.pyplot as plt
from fibsem import utils, conversions, patterning, milling, acquire
from fibsem.ui.utils import _draw_milling_stages_on_image
from fibsem import config as fcfg
from fibsem.imaging import _tile
from fibsem.structures import ImageSettings, BeamType, FibsemImage
from autolamella.structures import Experiment

# Steps:
# 1. Open AutoLamella, create a new experiment, load waffle protocol
# 2. Open Minimap, acquire Ion overview (max size 960um), select positions
# 3. Run this script to parallelize the trench milling

# paths
PATH = "/home/patrick/development/openfibsem/autolamella/autolamella/log/AutoLamella-2024-06-25-14-49"
EXPERIMENT_PATH = os.path.join(PATH, "experiment.yaml")
PROTOCOL_PATH = os.path.join(PATH, "protocol.yaml")

microscope, settings = utils.setup_session(protocol_path=PROTOCOL_PATH)


# load the experiment
exp = Experiment.load(os.path.join(PATH, "experiment.yaml"))

# load the overview image
overview_image = FibsemImage.load(os.path.join(PATH, "overview-image.tif"))

# move to the overview image position
microscope.move_stage_absolute(overview_image.metadata.microscope_state.stage_position)

# get the positions from the experiment (trench positions)
positions = [position for position in exp.positions]
stage_positions = [pos.state.microscope_state.stage_position for pos in positions]
points = _tile._reproject_positions(overview_image, stage_positions)

# convert the points to microscope image coordinates
milling_points = []
for point in points:
    mp = conversions.image_to_microscope_image_coordinates(point, overview_image.data, overview_image.metadata.pixel_size.x)
    mp.name = point.name
    milling_points.append(mp)

# check the points are within the field of view
# get the maximum field of view
shape = overview_image.data.shape
MAX_HFW = 960e-6 # ION BEAM MAX HFW
MAX_VFW = MAX_HFW * (shape[0] / shape[1])
print("Maximum FOV: ", MAX_HFW, MAX_VFW)

# check if the points are outside the field of view

for point, mp in zip(points, milling_points):
    if mp.x > MAX_HFW or mp.x < -MAX_HFW:
        print(f"Point {point.name} is outside the field of view")
    if mp.y > MAX_VFW or mp.y < -MAX_VFW:
        print(f"Point {point.name} is outside the field of view")

# plot the stage positions on the overview image
plt.imshow(overview_image.data, cmap="gray")
for point, pos in zip(points, stage_positions):
    plt.plot(point[0], point[1], "ro",) #label=pos.name)
plt.legend()

plt.show()

print('MILLING POINTS')

milling_stages = []
for pos, mp, sp in zip(positions, milling_points, stage_positions):

    stages = patterning.get_milling_stages("trench", settings.protocol["milling"], point=mp)

    stages[0].name = f"Trench {mp.name}"

    milling_stages.extend(stages)

fig = _draw_milling_stages_on_image(overview_image, milling_stages)
plt.show()

# mill the stages
milling.mill_stages(microscope, milling_stages)

# mark each position as finished trench milling
# go to each trench position and acquire reference images, mark finished? basically do trench workflow just dont mill


for pos in positions:
    
    settings.image.path = pos.path

    # move to the position
    microscope.move_stage_absolute(pos.state.microscope_state.stage_position)

    # acquire reference images
    reference_images = acquire.take_set_of_reference_images(
        microscope=microscope,
        image_settings=settings.image,
        hfws=[fcfg.REFERENCE_HFW_MEDIUM, fcfg.REFERENCE_HFW_HIGH],
        filename=f"ref_{pos.state.stage.name}_final",
    )


# ASSUMPTION: 
# all trenchs fit within a single max fov
# all trenches use the same milling parameters

# IMPROVEMENTS:
# Dont change current back down between milling stages - Done in script, not UI
# convert the milling points into the proper protocol format for consistency
# Implement in the user interface
# automatic save (ready) when selecting from minimap